# Data Cleaning

The data collected in the first step contains various types of noise. It's important to remove the noise to produce appropriate document embeddings. In order to clean the data, a series of regular expressions are used in this notebook.

In [ ]:
# Suppress FutureWarning messages
# PS: to suppress these messages it's necessary to keep this at the beginning of the notebook
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
# Versions used:
#
# python     3.9.7
# emoji      1.6.1
# matplotlib 3.5.0
# pandas     1.3.5

import datetime as dt
import html
import re

import emoji
import matplotlib.pyplot as plt
import pandas as pd

First, we need to load the data.

In [ ]:
# Read the DataFrame that was saved in the first notebook
posts_df = pd.read_csv('./rcollege_20200101-20220101_praw.csv')

# Return the first 5 rows
posts_df.head(5)

Then, we reorganize the data and remove unneeded rows/columns.

In [ ]:
# Keep only relevant columns
posts_df = posts_df[['id', 'created_utc', 'title', 'selftext', 'author', 'score']]

posts_df.head(5)

In [ ]:
posts_df['created_utc'] = pd.to_datetime(posts_df['created_utc'], unit='s')
posts_df['created_utc'] = posts_df['created_utc'].dt.date

posts_df['created_utc'].head(5)

In [ ]:
# Describe the data
posts_df.describe(include='all')

In [ ]:
# Remove posts with duplicate ids
posts_df = posts_df.drop_duplicates(subset='id')

# Remove posts that contain NA
posts_df = posts_df.dropna(subset=['id'])
posts_df = posts_df.dropna(subset=['created_utc'])
posts_df = posts_df.dropna(subset=['title'])
posts_df = posts_df.dropna(subset=['selftext'])
posts_df = posts_df.dropna(subset=['author'])
posts_df = posts_df.dropna(subset=['score'])

# Remove posts where author is [deleted]
posts_df = posts_df[posts_df.author != '[deleted]']

# Remove posts where title is [deleted by user]
posts_df = posts_df[posts_df.title != '[deleted by user]']

# Remove posts where selftext is [deleted] or [removed]
posts_df = posts_df[posts_df.selftext != '[deleted]']
posts_df = posts_df[posts_df.selftext != '[removed]']

# Remove posts where title and selftext are both duplicates
posts_df.drop_duplicates(subset=['title', 'selftext'], inplace=True)

posts_df.describe(include='all')

In [ ]:
# Convert dtypes
posts_df.convert_dtypes().dtypes

Afterwards, we can remove polls, unescape HTML entities and remove emojis.

In [ ]:
# Remove polls
posts_df['selftext'] = posts_df['selftext'].map(
    lambda x: re.sub(r'\[removed\]\s+\[View Poll\]\(https:\/\/www.reddit.com\/poll\/[A-Za-z0-9]+\)', '', x))
posts_df['selftext'] = posts_df['selftext'].map(
    lambda x: re.sub(r'\[View Poll\]\(https:\/\/www.reddit.com\/poll\/[A-Za-z0-9]+\)', '', x))

posts_df.describe(include='all')

In [ ]:
# Remove posts where selftext is \n\n
posts_df = posts_df[posts_df.selftext != '\n\n']

posts_df.describe(include='all')

In [ ]:
# Unescape HTML entities in title
posts_df['title'] = posts_df['title'].map(lambda x: html.unescape(x))

# Unescape HTML entities in selftext
# PS: when &amp;amp;amp;#x200B; is present it's necessary to run this 4 times
for _ in range(3):
    posts_df['selftext'] = posts_df['selftext'].map(lambda x: html.unescape(x))

# Remove remaining &#x200B;
posts_df['selftext'] = posts_df['selftext'].map(lambda x: re.sub('&#x200B;', '', x))

In [ ]:
# Remove emojis
posts_df['title'] = posts_df['title'].map(lambda x: re.sub(emoji.get_emoji_regexp(), ' ', x))
posts_df['selftext'] = posts_df['selftext'].map(lambda x: re.sub(emoji.get_emoji_regexp(), ' ', x))

posts_df.describe(include='all')

A series of regular expressions are then called to clean Markdown syntax and other types of noise.

In [ ]:
# Remove newlines
# posts_df['selftext'] = posts_df['selftext'].map(lambda x: re.sub(r'\n+', ' ', x).strip())
# posts_df['selftext'] = posts_df['selftext'].map(lambda x: re.sub(r'\s\s+', ' ', x))

# Remove syntax for quote block
posts_df['selftext'] = posts_df['selftext'].map(lambda x: re.sub(r'^\\>', '', x, flags=re.MULTILINE))

# Remove syntax for heading
posts_df['selftext'] = posts_df['selftext'].map(lambda x: re.sub(r'^#\s|^##\s', '', x, flags=re.MULTILINE))

# Replace \# in the beginning of a line with #
posts_df['selftext'] = posts_df['selftext'].map(lambda x: re.sub(r'^\\#', '#', x, flags=re.MULTILINE))

# Replace \* with a temporary code
# Where code = (asterisk + random combination of 8 characters)
posts_df['selftext'] = posts_df['selftext'].map(lambda x: re.sub(r'\\\*', '(asterisk=s&RDaeK)', x))

# Remove syntax for bold, italic and pointed list
posts_df['selftext'] = posts_df['selftext'].map(lambda x: re.sub(r'\*', '', x))

# Replace the temporary code above with *
posts_df['selftext'] = posts_df['selftext'].map(lambda x: re.sub(r'\(asterisk=s&RDaeK\)', '*', x))

# Remove space after open parenthesis or before close parenthesis
posts_df['title'] = posts_df['title'].map(lambda x: re.sub(r'\( ', '(', x))
posts_df['title'] = posts_df['title'].map(lambda x: re.sub(r' \)', ')', x))
posts_df['selftext'] = posts_df['selftext'].map(lambda x: re.sub(r'\( ', '(', x))
posts_df['selftext'] = posts_df['selftext'].map(lambda x: re.sub(r' \)', ')', x))

# Remove space after open bracket or before close bracket
posts_df['title'] = posts_df['title'].map(lambda x: re.sub(r'\[ ', '[', x))
posts_df['title'] = posts_df['title'].map(lambda x: re.sub(r' \]', ']', x))
posts_df['selftext'] = posts_df['selftext'].map(lambda x: re.sub(r'\[ ', '[', x))
posts_df['selftext'] = posts_df['selftext'].map(lambda x: re.sub(r' \]', ']', x))

# Remove clickable e-mails
posts_df['selftext'] = posts_df['selftext'].map(
    lambda x: re.sub(r'\[\]\(mailto:([A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,})\)', '', x))
posts_df['selftext'] = posts_df['selftext'].map(
    lambda x: re.sub(r'\["*?([A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,})\]\(mailto:"*?([A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,})\)', '', x))

# Remove clickable links, while mantaining the text
posts_df['title'] = posts_df['title'].map(
    lambda x: re.sub(r'[^0-9\D]?\[([^\[\s].*?)?\]\(([A-Za-z0-9]*?(:\/*)?(www[0-9]?\.)?[A-Za-z0-9\-]*(\.[A-Za-z0-9%-_]*)?\.[A-Za-z0-9]+\/?(\/[#-_a-~À-ÖØ-öø-ÿ™]+)*)\)[^0-9\D]?', r'\1', x))
posts_df['selftext'] = posts_df['selftext'].map(
    lambda x: re.sub(r'[^0-9\D]?\[([^\[\s].*?)?\]\(([A-Za-z0-9]*?(:\/*)?(www[0-9]?\.)?[A-Za-z0-9\-]*(\.[A-Za-z0-9%-_]*)?\.[A-Za-z0-9]+\/?(\/[#-_a-~À-ÖØ-öø-ÿ™]+)*)\)[^0-9\D]?', r'\1', x))

# Remove syntax for strikethrough
posts_df['selftext'] = posts_df['selftext'].map(lambda x: re.sub('~~(.*?)~~', r'\1', x))

# Remove links
posts_df['selftext'] = posts_df['selftext'].map(
    lambda x: re.sub(r'[A-Za-z0-9]+:\/\/[A-Za-z0-9%-_]+(\/[A-Za-z0-9%-_])*(#|\\?)[#-_a-~À-ÖØ-öø-ÿ™]*[^)]', '', x))

# Remove e-mails
posts_df['title'] = posts_df['title'].map(
    lambda x: re.sub('([A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,})', '', x))
posts_df['selftext'] = posts_df['selftext'].map(
    lambda x: re.sub('([A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,})', '', x))

# Remove syntax for spoiler
posts_df['selftext'] = posts_df['selftext'].map(lambda x: re.sub('>!(.*?)!<', r'\1', x))

# Remove syntax for inline code and code block
posts_df['selftext'] = posts_df['selftext'].map(lambda x: re.sub('`', '', x))

# Remove emotes :-) :-( :-/
posts_df['selftext'] = posts_df['selftext'].map(lambda x: re.sub(r':-\)|:-\(|:-\/', '', x))

# Remove emote T^T
posts_df['selftext'] = posts_df['selftext'].map(lambda x: re.sub(r'T\^T', '', x))

# Remove syntax for superscript
posts_df['selftext'] = posts_df['selftext'].map(lambda x: re.sub(r'\^\((.*?)\)', r'\1', x))
posts_df['selftext'] = posts_df['selftext'].map(lambda x: re.sub(r'\^', '', x))

# Remove syntax for table
posts_df['selftext'] = posts_df['selftext'].map(lambda x: re.sub(r'\|', ' ', x))
posts_df['selftext'] = posts_df['selftext'].map(lambda x: re.sub(':-', '', x))

# Replace \[ with [ and \] with ]
posts_df['selftext'] = posts_df['selftext'].map(lambda x: re.sub(r'\\\[', '[', x))
posts_df['selftext'] = posts_df['selftext'].map(lambda x: re.sub(r'\\\]', ']', x))

# Replace multiple spaces with a single space
posts_df['selftext'] = posts_df['selftext'].map(lambda x: re.sub(' +', ' ', x).strip())

posts_df.describe(include='all')

Finally, the leftover noise can be removed.

In [ ]:
# Remove posts with empty selftext
posts_df = posts_df[posts_df.selftext != '']
posts_df = posts_df[posts_df.selftext != '\u200b']

# Remove posts where selftext is [deleted] or [removed]
# posts_df = posts_df[posts_df.selftext != '[deleted]']
# posts_df = posts_df[posts_df.selftext != '[removed]']

# Return the most common value for selftext
# This is useful for finding entities such as \u200b
# posts_df['selftext'].value_counts().idxmax()

posts_df.describe(include='all')

Now, the data will be reorganized and stored in a new CSV file.

In [ ]:
# For each row of the DataFrame, join title and selftext
# We'll call this column "document"
posts_df['document'] = posts_df['title'] + '\n' + posts_df['selftext']

# Keep only the columns created_utc, document and score
posts_df = posts_df[['created_utc', 'document', 'score']]

posts_df.head(5)

In [ ]:
# Save the cleaned data
posts_df.to_csv('./rcollege_clean_20200101-20220101_praw.csv', columns=list(posts_df.axes[1]), header=True, index=False)

Lastly, we can visualize the approximate number of tokens per document.

In [ ]:
"""
plt_x = posts_df['document'].str.split().map(lambda x: len(x))

plt.figure(figsize=(12, 6))

plt.xlabel("Número de $\it{Tokens}$", fontsize=10)
plt.ylabel("Número de Documentos", fontsize=10)

plt.hist(plt_x, color='xkcd:bluey purple', bins=range(0, 768))

plt.axvline(plt_x.mean(), color='k', linestyle='dashed', linewidth=1)

min_ylim, max_ylim = plt.ylim()
plt.text(plt_x.mean()*1.1, max_ylim*0.9, "Média: {:.2f}".format(plt_x.mean()))
"""

plt_x = posts_df['document'].str.split().map(lambda x: len(x))

plt.figure(figsize=(12, 6))

plt.xlabel("Number of Tokens", fontsize=10)
plt.ylabel("Number of Documents", fontsize=10)

plt.hist(plt_x, color='xkcd:bluey purple', bins=range(0, 768))

plt.axvline(plt_x.mean(), color='k', linestyle='dashed', linewidth=1)

min_ylim, max_ylim = plt.ylim()
plt.text(plt_x.mean()*1.1, max_ylim*0.9, "Mean: {:.2f}".format(plt_x.mean()))

In [ ]:
j0 = [i for i in list(plt_x) if i <= 128]
p0 = 100 * len(j0)/len(plt_x)

j1 = [i for i in list(plt_x) if i <= 256]
p1 = 100 * len(j1)/len(plt_x)

j2 = [i for i in list(plt_x) if i <= 384]
p2 = 100 * len(j2)/len(plt_x)

j3 = [i for i in list(plt_x) if i <= 512]
p3 = 100 * len(j3)/len(plt_x)

print("{0:.2f} percent of the documents contain up to 128 tokens.".format(p0))
print("{0:.2f} percent of the documents contain up to 256 tokens.".format(p1))
print("{0:.2f} percent of the documents contain up to 384 tokens.".format(p2))
print("{0:.2f} percent of the documents contain up to 512 tokens.".format(p3))